## Simple Collision avoidance

In [1]:
import numpy as np

# Custom modules

from helpers import global2local, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC
from plan import Plan, State
from oracle import Oracle

kill_processes()

## Create Plans

In [2]:
offsets = [(-10, 0, 0, 0),(10, 0, 0, 0)]#
n_vehicles=len(offsets)


homes=np.array([offset[:3] for offset in offsets])
global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
              np.array([[10, 0, 5],[-10, 0, 5]])]
local_paths=[global2local(path, home) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5,navegation_speed=1) for path in local_paths]

## Visualization Parameters

In [3]:
waypoints = {f'waypoint_{i+1}': {'pos':global_paths[0],'color':'blue'} for i in range(n_vehicles)}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)

## Choose Simulator

In [4]:
# simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path='simulators/gazebo/worlds/runway.world',
                 vehicle_models=n_vehicles*['drone'],
                 markers=waypoints)

Launch Simulator

In [5]:
uavs = simulator.launch()
orac = Oracle(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀


## Execute Plan

In [ ]:
while orac.vehs:
    orac.gather_broadcasts()
    for uav in orac.vehs.copy():
        orac.update_neighbors(uav)
        if uav.plan.state == State.DONE:
            orac.remove(uav)
        else:
            uav.act()
kill_processes()

In [ ]:
uavs[0].plan

✅ <Plan 'basic'>
  ✅ <Action 'PREARM'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'MODE: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'PARAM: nav speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'ARM'>
    ✅ <Step 'arm'>
  ✅ <Action 'TAKEOFF'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'FLY'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (8.07105896157324, 5.022533236403946, 4.87105655670166)'>
    ✅ <Step 'go to  -> (20, 0, 5)'>
  ✅ <Action 'LAND'>
    ✅ <Step 'land'>

In [ ]:
uavs[1].plan

✅ <Plan 'basic'>
  ✅ <Action 'PREARM'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'MODE: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'PARAM: nav speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'ARM'>
    ✅ <Step 'arm'>
  ✅ <Action 'TAKEOFF'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'FLY'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (-8.046924325647947, -5.023265060369853, 4.914658546447754)'>
    ✅ <Step 'go to  -> (-20, 0, 5)'>
  ✅ <Action 'LAND'>
    ✅ <Step 'land'>